# Unit commitment for a simple power grid
This example shows how pyoframe is used to define and solve an optimization problem, comparing step-by-step with [gurobipy](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html). The problem is a highly simplified version of the unit commitment problem that is solved daily in a large number of electrical markets around the world.

<img src="./three-bus.png" alt="Three-bus grid" width="500">

### Problem Setup

**Objective**: Minimize the total cost of electricity generation.  
**Constraints**: Meet demand at each bus, respect generation limits, handle binary on/off status of each generator, and account for transmission limits between buses.

## Implementation

In [17]:
%pip install pyoframe gurobipy pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import gurobipy as gp
import pandas as pd
import polars as pl
from gurobipy import GRB

import pyoframe as pf

### Input data

### Parameters

**Costs**
- $c_g(\text{gen})$: Cost of generation per unit of power from generator $\text{gen}$ (\$/MW).
- $c_{su}(\text{gen})$: Cost of starting up generator $\text{gen}$ (\$).

**Technical Limits**
- $\text{min}P_g$, $\text{max}P_g$: Minimum and maximum power output of generator $\text{gen}$ (MW).
- $D_{n,t}$: Demand at bus $n$ at time $t$ (MW).

**Transmission Parameters**
- $\text{max}P_l$: Maximum capacity of transmission line $l$ (MW).
- $B_l = B_{jk}$: Susceptance of line $l$ between bus $j$ and bus $k$.
- $A_{jk}$: Entry of incidence matrix $A$ (takes values $1$ or $-1$ depending on the direction of the flow between bus $j$ and bus $k$ for line $l$).

**Initial State**
- $I(\text{gen},0)$: Generator $\text{gen}$ is initially on or off.

In [19]:
generators = pl.DataFrame(
    [
        {
            "gen": 1,
            "bus": 1,
            "I_init": 1,
            "min_power": 50,
            "max_power": 150,
            "cost": 25,
            "start_up_cost": 200,
        },
        {
            "gen": 2,
            "bus": 2,
            "I_init": 0,
            "min_power": 30,
            "max_power": 100,
            "cost": 30,
            "start_up_cost": 150,
        },
        {
            "gen": 3,
            "bus": 3,
            "I_init": 0,
            "min_power": 20,
            "max_power": 80,
            "cost": 50,
            "start_up_cost": 100,
        },
    ]
)

transmission_lines = pl.DataFrame(
    [
        {"l": 1, "from": 1, "to": 2, "capacity": 60, "susceptance": 20},
        {"l": 2, "from": 2, "to": 3, "capacity": 50, "susceptance": 15},
        {"l": 3, "from": 1, "to": 3, "capacity": 40, "susceptance": 10},
    ]
)

load = pl.DataFrame(
    {
        "t": [1, 2, 3] * 3,
        "bus": [1] * 3 + [2] * 3 + [3] * 3,
        "demand": [100, 120, 30, 50, 110, 25, 40, 90, 15],
    }
)

time_periods = load["t"].unique().sort()
busses = pd.DataFrame({"n": [1, 2, 3]})
generators.to_pandas()

,gen,bus,I_init,min_power,max_power,cost,start_up_cost
0,1,1,1,50,150,25,200
1,2,2,0,30,100,30,150
2,3,3,0,20,80,50,100


In [20]:
load.sort("t").to_pandas()

,t,bus,demand
0,1,1,100
1,1,2,50
2,1,3,40
3,2,1,120
4,2,2,110
5,2,3,90
6,3,1,30
7,3,2,25
8,3,3,15


### Instantiating the model

#### Gurobi

In [21]:
gp_model = gp.Model("Three-bus example")

#### PyoFrame

In [22]:
pf_model = pf.Model()

### Defining variables

### Variables

**Binary Decision Variables**
- $I(\text{gen},t)$: Generator $\text{gen}$ is on ($I(\text{gen},t) = 1$) or off ($I(\text{gen},t) = 0$) at time $t$.
- $SU(\text{gen},t)$: Generator $\text{gen}$ starts up ($SU(\text{gen},t) = 1$) at time $t$.
- $SD(\text{gen},t)$: Generator $\text{gen}$ shuts down ($SD(\text{gen},t) = 1$) at time $t$.

**Continuous Decision Variables**
- $P_g(\text{gen},t)$: Power generated by generator $\text{gen}$ at time $t$ (MW).
- $P_l(l,t)$: Power flow on transmission line $l$ at time $t$ (MW).
- $\delta_{n,t}$: Voltage angle at bus $n$ at time $t$ (centi-radians).

#### Gurobi

In [ ]:
I = gp_model.addVars(generators["gen"], time_periods, vtype=GRB.BINARY, name="I")  # noqa: E741
SU = gp_model.addVars(generators["gen"], time_periods, vtype=GRB.BINARY, name="SU")
SD = gp_model.addVars(generators["gen"], time_periods, vtype=GRB.BINARY, name="SD")
P_g = gp_model.addVars(
    generators["gen"], time_periods, vtype=GRB.CONTINUOUS, name="P_g"
)
P_l = gp_model.addVars(
    transmission_lines["l"],
    time_periods,
    lb=-float("inf"),
    vtype=GRB.CONTINUOUS,
    name="P_l",
)
delta_n = gp_model.addVars(
    busses["n"], time_periods, vtype=GRB.CONTINUOUS, lb=-float("inf"), name="delta_n"
)

#### PyoFrame

In [24]:
pf_model.I = pf.Variable(generators["gen"], time_periods, vtype=pf.VType.BINARY)
pf_model.SU = pf.Variable(generators["gen"], time_periods, vtype=pf.VType.BINARY)
pf_model.SD = pf.Variable(generators["gen"], time_periods, vtype=pf.VType.BINARY)
pf_model.P_g = pf.Variable(generators["gen"], time_periods, lb=0)
pf_model.P_l = pf.Variable(transmission_lines["l"], time_periods)
pf_model.delta_n = pf.Variable(busses, time_periods)

### Objective Function

Minimize the total cost:

$$
\min \sum_{t} \left(\sum_{\text{gen}} \left( c_g(\text{gen}) P_g(\text{gen},t) + c_{su}(\text{gen}) SU(\text{gen},t) \right) \right)
$$

#### Gurobi

In [25]:
generation_cost = gp.quicksum(
    generators.filter(gen=i)["cost"].item() * P_g[i, t]
    for i in generators["gen"]
    for t in time_periods
)
start_up_cost = gp.quicksum(
    generators.filter(gen=i)["start_up_cost"].item() * SU[i, t]
    for i in generators["gen"]
    for t in time_periods
)

gp_model.setObjective(generation_cost + start_up_cost, GRB.MINIMIZE)

#### PyoFrame

In [26]:
pf_model.minimize = pf.sum(
    ["t", "gen"], pf_model.P_g * generators.select("gen", "cost")
)
pf_model.minimize += pf.sum(
    "t", pf.sum("gen", pf_model.SU * generators.select("gen", "start_up_cost"))
)

In [27]:
pf_model.objective

<Objective size=1 dimensions={} terms=18>
objective: 25 P_g[1,1] +25 P_g[1,2] +25 P_g[1,3] +30 P_g[2,1] +30 P_g[2,2] +30 P_g[2,3] +50...

## Constraints

### Power balance


1. **Power Balance at Each Bus**  
   For each bus $n$ and time $t$:

   $$
   \sum_{\text{gen} \in G_n} P_g(\text{gen},t) - D(n,t) = \sum_{k} A_{nk} P_l(k,t)
   $$

   where $G_n$ is the set of generators connected to bus $n$.

#### Gurobi

In [ ]:
for t in time_periods:
    for bus in busses["n"]:
        demand = load.filter(bus=bus, t=t)["demand"].item()
        total_power_gen = gp.quicksum(
            P_g[gen, t] for gen in generators.filter(bus=bus)["gen"]
        )
        total_power_flow_to = gp.quicksum(
            P_l[line, t] for line in transmission_lines.filter(to=bus)["l"]
        )
        total_power_flow_from = gp.quicksum(
            P_l[line, t]
            for line in transmission_lines.filter(pl.col("from").eq(bus))["l"]
        )

        gp_model.addConstr(
            total_power_gen - demand == total_power_flow_to - total_power_flow_from
        )

#### PyoFrame

In [29]:
pf_model.P_l

<Variable name=P_l size=9 dimensions={'l': 3, 't': 3}>
[1,1]: P_l[1,1]
[1,2]: P_l[1,2]
[1,3]: P_l[1,3]
[2,1]: P_l[2,1]
[2,2]: P_l[2,2]
[2,3]: P_l[2,3]
[3,1]: P_l[3,1]
[3,2]: P_l[3,2]
[3,3]: P_l[3,3]

In [30]:
# Power flow to and from each bus
P_l_from = pf_model.P_l.map(
    transmission_lines.select(["l", "from"]).rename({"from": "bus"})
)
P_l_to = pf_model.P_l.map(transmission_lines.select(["l", "to"]).rename({"to": "bus"}))
P_gen_at_bus = pf_model.P_g.map(generators.select("gen", "bus"))

# Power Balance at Each Bus
pf_model.PowBal = P_gen_at_bus - load == P_l_from.keep_extras() - P_l_to.keep_extras()

In [31]:
pf_model.PowBal

<Constraint name=PowBal sense='=' size=9 dimensions={'t': 3, 'bus': 3} terms=36>
[1,1]: - P_l[1,1] - P_l[3,1] + P_g[1,1] = 100
[1,2]: - P_l[2,1] + P_l[1,1] + P_g[2,1] = 50
[1,3]: P_l[2,1] + P_l[3,1] + P_g[3,1] = 40
[2,1]: - P_l[1,2] - P_l[3,2] + P_g[1,2] = 120
[2,2]: - P_l[2,2] + P_l[1,2] + P_g[2,2] = 110
[2,3]: P_l[2,2] + P_l[3,2] + P_g[3,2] = 90
[3,1]: - P_l[1,3] - P_l[3,3] + P_g[1,3] = 30
[3,2]: - P_l[2,3] + P_l[1,3] + P_g[2,3] = 25
[3,3]: P_l[2,3] + P_l[3,3] + P_g[3,3] = 15

### Generation limits

2. **Generation Limits**  
   For each generator $\text{gen}$ and time $t$:

   $$
   \text{min}P_g \cdot I(\text{gen},t) \leq P_g(\text{gen},t) \leq \text{max}P_g \cdot I(\text{gen},t)
   $$

#### Gurobi

In [32]:
for gen in generators["gen"]:
    for t in time_periods:
        gp_model.addConstr(
            P_g[gen, t] >= generators.filter(gen=gen)["min_power"].item() * I[gen, t]
        )
        gp_model.addConstr(
            P_g[gen, t] <= generators.filter(gen=gen)["max_power"].item() * I[gen, t]
        )

#### PyoFrame

In [33]:
pf_model.re_P_g_maxgen = (
    pf_model.P_g <= generators.select("gen", "max_power") * pf_model.I
)
pf_model.re_P_g_mingen = (
    pf_model.P_g >= generators.select("gen", "min_power") * pf_model.I
)

### Generator on/off dynamics

3. **Generator On/Off Dynamics**  
   For each generator $\text{gen}$ and time $t$:

   $$
   I(\text{gen},t) = I(\text{gen}, t-1) + SU(\text{gen},t) - SD(\text{gen},t)
   $$

#### Gurobi

In [34]:
I0 = generators.select("gen", "I_init")
for gen in generators["gen"]:
    gp_model.addConstr(
        I[gen, 1] == I0.filter(gen=gen)["I_init"].item() + SU[gen, 1] - SD[gen, 1]
    )
    for t in time_periods[1:]:
        gp_model.addConstr(I[gen, t] == I[gen, t - 1] + SU[gen, t] - SD[gen, t])

#### PyoFrame

In [35]:
I0 = generators.select("gen", "I_init").to_expr().over("t")
pf_model.eq_I_init = pf_model.I.filter(t=1) == I0 + pf_model.SU.filter(
    t=1
) - pf_model.SD.filter(t=1)
pf_model.eq_I = pf_model.I.next("t") - pf_model.I.drop_extras() == pf_model.SU.next(
    "t"
) - pf_model.SD.next("t")
pf_model.eq_I_init, pf_model.eq_I

(<Constraint name=eq_I_init sense='=' size=3 dimensions={'gen': 3, 't': 1} terms=10>
 [1,1]: I[1,1] - SU[1,1] + SD[1,1] = 1
 [2,1]: I[2,1] - SU[2,1] + SD[2,1] = 0
 [3,1]: I[3,1] - SU[3,1] + SD[3,1] = 0,
 <Constraint name=eq_I sense='=' size=6 dimensions={'gen': 3, 't': 2} terms=24>
 [1,1]: - I[1,1] + I[1,2] - SU[1,2] + SD[1,2] = 0
 [1,2]: - I[1,2] + I[1,3] - SU[1,3] + SD[1,3] = 0
 [2,1]: - I[2,1] + I[2,2] - SU[2,2] + SD[2,2] = 0
 [2,2]: - I[2,2] + I[2,3] - SU[2,3] + SD[2,3] = 0
 [3,1]: - I[3,1] + I[3,2] - SU[3,2] + SD[3,2] = 0
 [3,2]: - I[3,2] + I[3,3] - SU[3,3] + SD[3,3] = 0)

### Transmission limits

4. **Transmission Line Capacity Limits**  
   For each transmission line $l$ and time $t$:

   $$
   -\text{max}P_l \leq P_l(l, t) \leq \text{max}P_l
   $$

#### Gurobi

In [ ]:
for line in transmission_lines["l"]:
    for t in time_periods:
        line_limit = transmission_lines.filter(l=line)["capacity"].item()
        gp_model.addConstr(P_l[line, t] <= line_limit)
        gp_model.addConstr(P_l[line, t] >= -line_limit)

#### PyoFrame

In [37]:
pf_model.re_Pl_FT = pf_model.P_l <= transmission_lines.select(
    "l", "capacity"
).to_expr().over("t")
pf_model.re_Pl_TF = pf_model.P_l >= -transmission_lines.select(
    "l", "capacity"
).to_expr().over("t")

### DC Power flow

5. **DC Power Flow**  
   The power flow on each transmission line $l$ from bus $j$ to bus $k$ at time $t$ is given by:

   $$
   P_l(l,t) = B_l \left(\delta_n(j,t) - \delta_n(k,t)\right)
   $$

#### Gurobi

In [ ]:
for line in transmission_lines["l"]:
    for t in time_periods:
        from_bus = transmission_lines.filter(l=line)["from"].item()
        to_bus = transmission_lines.filter(l=line)["to"].item()
        susc = transmission_lines.filter(l=line)["susceptance"].item()
        gp_model.addConstr(
            P_l[line, t] == susc * (delta_n[from_bus, t] - delta_n[to_bus, t])
        )

#### PyoFrame

In [39]:
pf_model.delta_n

<Variable name=delta_n size=9 dimensions={'n': 3, 't': 3}>
[1,1]: delta_n[1,1]
[1,2]: delta_n[1,2]
[1,3]: delta_n[1,3]
[2,1]: delta_n[2,1]
[2,2]: delta_n[2,2]
[2,3]: delta_n[2,3]
[3,1]: delta_n[3,1]
[3,2]: delta_n[3,2]
[3,3]: delta_n[3,3]

In [40]:
voltage_angle_from = pf_model.delta_n.map(
    transmission_lines.select("l", "from").rename({"from": "n"})
)
voltage_angle_to = pf_model.delta_n.map(
    transmission_lines.select("l", "to").rename({"to": "n"})
)
susc = transmission_lines.select("l", "susceptance")
pf_model.eq_dc = pf_model.P_l == susc * (voltage_angle_from - voltage_angle_to)

In [41]:
voltage_angle_from

<Expression size=9 dimensions={'t': 3, 'l': 3} terms=9>
[1,1]: delta_n[1,1]
[1,3]: delta_n[1,1]
[2,1]: delta_n[1,2]
[2,3]: delta_n[1,2]
[3,1]: delta_n[1,3]
[3,3]: delta_n[1,3]
[1,2]: delta_n[2,1]
[2,2]: delta_n[2,2]
[3,2]: delta_n[2,3]

### Voltage angle reference

6. **Voltage Angle Reference**  
   To ensure a unique solution, fix the voltage angle at one bus (e.g., $\delta_n(1,t) = 0$).

#### Gurobi

In [42]:
for t in time_periods:
    gp_model.addConstr(delta_n[1, t] == 0)

#### PyoFrame

In [43]:
pf_model.eq_slack = pf_model.delta_n.filter(n=1) == 0

## Solve the model

#### Gurobi

In [44]:
gp_model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 66 rows, 54 columns and 144 nonzeros
Model fingerprint: 0xb7de586e
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 59 rows and 48 columns
Presolve time: 0.08s
Presolved: 7 rows, 6 columns, 21 nonzeros
Variable types: 3 continuous, 3 integer (3 binary)
Found heuristic solution: objective 18950.000000
Found heuristic solution: objective 17200.000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.12 seconds (0.00 work units)
Thread count was 8 (of 8 available proces

In [ ]:
def print_gurobi_solution():
    m = gp_model
    if m.status == GRB.OPTIMAL:
        print(f"Optimal solution found! Best objective {m.objVal}")
        for t in time_periods:
            for gen in generators["gen"]:
                print(
                    f"Generator {gen} at time {t}: On/Off = {I[gen, t].x}, Power Output = {P_g[gen, t].x} MW"
                )
        for t in time_periods:
            for line in transmission_lines["l"]:
                print(
                    f"Transmission Line {line} at time {t}: Power Flow = {P_l[line, t].x} MW"
                )
    else:
        print("No optimal solution found.")


print_gurobi_solution()

Optimal solution found! Best objective 17200.0
Generator 1 at time 1: On/Off = 1.0, Power Output = 150.0 MW
Generator 2 at time 1: On/Off = 1.0, Power Output = 39.99999999999999 MW
Generator 3 at time 1: On/Off = -0.0, Power Output = 0.0 MW
Generator 1 at time 2: On/Off = 1.0, Power Output = 150.0 MW
Generator 2 at time 2: On/Off = 1.0, Power Output = 100.0 MW
Generator 3 at time 2: On/Off = 1.0, Power Output = 70.0 MW
Generator 1 at time 3: On/Off = 1.0, Power Output = 70.0 MW
Generator 2 at time 3: On/Off = -0.0, Power Output = 0.0 MW
Generator 3 at time 3: On/Off = -0.0, Power Output = 0.0 MW
Transmission Line 1 at time 1: Power Flow = -26.153846153846153 MW
Transmission Line 2 at time 1: Power Flow = -16.153846153846153 MW
Transmission Line 3 at time 1: Power Flow = -23.846153846153847 MW
Transmission Line 1 at time 2: Power Flow = -16.923076923076906 MW
Transmission Line 2 at time 2: Power Flow = -6.923076923076913 MW
Transmission Line 3 at time 2: Power Flow = -13.076923076923077

#### PyoFrame

In [47]:
result = pf_model.optimize()

In [48]:
def print_pyoframe_solution():
    m = pf_model
    print("Dispatch:")
    print(m.I.solution.sort("t"))
    print("Generation:")
    print(m.P_g.solution.sort("t"))


print_pyoframe_solution()

Dispatch:
shape: (9, 3)
┌─────┬─────┬──────────┐
│ gen ┆ t   ┆ solution │
│ --- ┆ --- ┆ ---      │
│ i64 ┆ i64 ┆ i64      │
╞═════╪═════╪══════════╡
│ 1   ┆ 1   ┆ 1        │
│ 2   ┆ 1   ┆ 1        │
│ 3   ┆ 1   ┆ 0        │
│ 1   ┆ 2   ┆ 1        │
│ 2   ┆ 2   ┆ 1        │
│ 3   ┆ 2   ┆ 1        │
│ 1   ┆ 3   ┆ 1        │
│ 2   ┆ 3   ┆ 0        │
│ 3   ┆ 3   ┆ 0        │
└─────┴─────┴──────────┘
Generation:
shape: (9, 3)
┌─────┬─────┬──────────┐
│ gen ┆ t   ┆ solution │
│ --- ┆ --- ┆ ---      │
│ i64 ┆ i64 ┆ f64      │
╞═════╪═════╪══════════╡
│ 1   ┆ 1   ┆ 150.0    │
│ 2   ┆ 1   ┆ 40.0     │
│ 3   ┆ 1   ┆ 0.0      │
│ 1   ┆ 2   ┆ 150.0    │
│ 2   ┆ 2   ┆ 100.0    │
│ 3   ┆ 2   ┆ 70.0     │
│ 1   ┆ 3   ┆ 70.0     │
│ 2   ┆ 3   ┆ 0.0      │
│ 3   ┆ 3   ┆ 0.0      │
└─────┴─────┴──────────┘
